In [1]:
import pydicom as dicom
import cv2
import numpy as np
import csv
import os

In [43]:
# read dicom and create blank mask
trainpath = '/sda1/zhouziyu/ssl/dataset/SIIM-ACR_Pneumothorax_segmentation/train/'
dst_img = '/sda1/zhouziyu/ssl/dataset/SIIM-ACR_Pneumothorax_segmentation/image_all/'
dst_mask = '/sda1/zhouziyu/ssl/dataset/SIIM-ACR_Pneumothorax_segmentation/mask_all/'
trainlist = os.listdir(trainpath)
for i in trainlist:
    ds = dicom.dcmread(trainpath+i)
    # img = np.array(ds.pixel_array)
    # cv2.imwrite(dst_img+i+'.jpeg', img)
    h = ds.Rows
    w = ds.Columns
    mask= np.zeros([h,w,3],dtype="uint8")
    cv2.imwrite(dst_mask+i+'_mask.png', mask)

In [46]:
# load csv label
dst_mask = '/sda1/zhouziyu/ssl/dataset/SIIM-ACR_Pneumothorax_segmentation/mask_all/'
traincsv = '/sda1/zhouziyu/ssl/dataset/SIIM-ACR_Pneumothorax_segmentation/stage_2_train.csv'
imglist = []

# decode mask
# def rle2mask(rle, width, height):
#     mask= np.zeros(width* height)
#     array = np.asarray([int(x) for x in rle.split()])
#     starts = array[0::2]
#     lengths = array[1::2]

#     current_position = 0
#     for index, start in enumerate(starts):
#         current_position += start
#         mask[current_position:current_position+lengths[index]] = 255
#         current_position += lengths[index]

#     return mask.reshape(width, height)

def rle2mask(rle, mask):
    h,w,_ = mask.shape
    mask= mask[:,:,0]
    mask = mask.reshape(-1)
    array = np.asarray([int(x) for x in rle.split()])
    starts = array[0::2]
    lengths = array[1::2]
    # print(np.unique(mask))

    current_position = 0
    for index, start in enumerate(starts):
        current_position += start
        mask[current_position:current_position+lengths[index]] = 255
        current_position += lengths[index]
    # print(np.unique(mask))
    mask = mask.reshape(w, h)
    mask = np.expand_dims(mask, axis=2)
    mask = np.repeat(mask, 3, axis=2)

    return mask

with open(traincsv, 'r') as fileDescriptor:
    csvReader = csv.reader(fileDescriptor)
    next(csvReader, None)
    for line in csvReader:
        if line[2]!='-1':
            imgname = line[1]
            mask = cv2.imread(dst_mask+imgname+'.dcm_mask.png')
            # print(np.unique(mask, return_counts=True))
            decode_mask = rle2mask(line[2], mask)
            # print(np.unique(decode_mask))
            # print(decode_mask.dtype)
            cv2.imwrite(dst_mask+imgname+'.dcm_mask.png', decode_mask)
            # break

In [9]:
# npy --> txt
testlist = np.load('/data/zhouziyu/home3/zhouziyu/warmup/sslpretrain/Swin-Transformer/data/data_split/SIIM/validation_image_mask.npy', allow_pickle=True)
print(testlist)
print(testlist.shape)
with open('/data/zhouziyu/home3/zhouziyu/warmup/sslpretrain/Swin-Transformer/data/data_split/SIIM/val.txt', 'w') as f:
    for i in range(testlist.shape[0]):
        f.writelines(testlist[i,0]+'\n')
f.close()

[['1.2.276.0.7230010.3.1.4.8323329.4541.1517875183.370160.dcm' ' -1']
 ['1.2.276.0.7230010.3.1.4.8323329.11747.1517875234.580518.dcm' ' -1']
 ['1.2.276.0.7230010.3.1.4.8323329.10374.1517875223.421330.dcm'
  ' 615576 14 1004 22 999 27 994 31 989 35 985 38 980 43 975 48 964 59 959 65 957 66 957 67 956 67 956 67 956 67 956 67 956 66 957 66 957 66 957 66 957 66 958 65 958 65 958 65 959 64 960 62 961 62 962 60 964 59 964 59 965 58 965 59 965 58 966 57 967 57 967 56 968 55 969 54 969 55 969 54 970 54 970 53 971 53 972 51 973 51 973 50 974 50 974 49 975 48 976 48 976 47 977 47 978 45 979 45 979 44 980 44 980 43 981 43 981 43 982 41 983 41 983 41 983 41 984 40 984 40 984 40 984 40 984 40 985 39 985 39 985 38 986 38 986 38 986 38 986 37 987 37 988 36 988 35 989 35 989 35 989 35 990 35 989 35 989 35 989 35 989 35 990 34 990 34 990 35 989 35 990 34 990 34 990 35 989 35 989 35 990 34 990 35 989 35 989 35 989 36 989 35 989 35 989 36 988 36 989 35 989 36 988 36 988 37 988 37 987 38 986 39 985 40 985

In [13]:
traintxt = '/data/zhouziyu/home3/zhouziyu/warmup/sslpretrain/Swin-Transformer/data/data_split/SIIM/val_new.txt'

with open('/data/zhouziyu/home3/zhouziyu/warmup/sslpretrain/Swin-Transformer/data/data_split/SIIM/val.txt', 'w') as f:
    with open(traintxt, encoding='utf-8') as e: # load train list and train label
        list = e.readlines()
        for i in list:
            i = i.split('\n')[0]
            train_list = i.split('.jpeg')[0]
            f.writelines(train_list+'\n')
f.close()


In [ ]:
# MontgomerySet
leftmask = '/sda1/zhouziyu/ssl/dataset/MontgomerySet/ManualMask/leftMask/'
rightmask = '/sda1/zhouziyu/ssl/dataset/MontgomerySet/ManualMask/rightMask/'
allmask = '/sda1/zhouziyu/ssl/dataset/MontgomerySet/ManualMask/LeftRightMask/'

masklist = os.listdir(leftmask)
for i in masklist:
    mask1 = cv2.imread(leftmask+i)
    mask2 = cv2.imread(rightmask+i)
    mask2[mask1==255]=255
    cv2.imwrite(allmask+i, mask2)